In [14]:
import pybullet as p
import pybullet_data
import argparse
import numpy as np
import os
import os.path as osp
import joblib
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm
import sys
from contextlib import contextmanager
from multiprocessing import Pool
import time
import plotly
import plotly.graph_objs as go
import json
import trimesh

In [101]:
# file_name = "data/parkour/motion/parkour-speed-vaultl.json"
file_name = "data/parkour/try_poses.json"
with open(file_name) as file:
    motion = json.load(file)
mesh_file_name = "data/parkour/object/Parkour_Speed Vault(L)/obj.obj"
mesh = trimesh.load(mesh_file_name)
vertices = mesh.vertices

In [102]:
vertices = mesh.vertices / 100
print(vertices)

[[ 0.43170593 -0.00209164 -0.08844433]
 [ 1.00228767 -0.0020919  -0.12930559]
 [ 1.00228767  0.72663563 -0.1293059 ]
 [ 0.43170593  0.72663574 -0.08844461]
 [ 0.39857309 -0.00209164 -0.55110512]
 [ 0.39857309  0.72663574 -0.55110542]
 [ 0.96915474 -0.0020919  -0.59196648]
 [ 0.96915474  0.72663563 -0.59196678]]


In [103]:
pelvis = np.array(motion["CC_Base_Pelvis"])

l_upperarm = np.array(motion["CC_Base_L_Upperarm"])
l_forearm = np.array(motion["CC_Base_L_Forearm"])
l_hand = np.array(motion["CC_Base_L_Hand"])

r_upperarm = np.array(motion["CC_Base_R_Upperarm"])
r_forearm = np.array(motion["CC_Base_R_Forearm"])
r_hand = np.array(motion["CC_Base_R_Hand"])

l_thigh = np.array(motion["CC_Base_L_Thigh"])
l_calf = np.array(motion["CC_Base_L_Calf"])
l_foot = np.array(motion["CC_Base_L_Foot"])
l_toe = np.array(motion["CC_Base_L_ToeBase"])

r_thigh = np.array(motion["CC_Base_R_Thigh"])
r_calf = np.array(motion["CC_Base_R_Calf"])
r_foot = np.array(motion["CC_Base_R_Foot"])
r_toe = np.array(motion["CC_Base_R_ToeBase"])

# pelvis = np.array(motion["CC_Base_Pelvis"])[:,[2,0,1]]

# l_upperarm = np.array(motion["CC_Base_L_Upperarm"])[:,[2,0,1]]
# l_forearm = np.array(motion["CC_Base_L_Forearm"])[:,[2,0,1]]
# l_hand = np.array(motion["CC_Base_L_Hand"])[:,[2,0,1]]

# r_upperarm = np.array(motion["CC_Base_R_Upperarm"])[:,[2,0,1]]
# r_forearm = np.array(motion["CC_Base_R_Forearm"])[:,[2,0,1]]
# r_hand = np.array(motion["CC_Base_R_Hand"])[:,[2,0,1]]

# l_thigh = np.array(motion["CC_Base_L_Thigh"])[:,[2,0,1]]
# l_calf = np.array(motion["CC_Base_L_Calf"])[:,[2,0,1]]
# l_foot = np.array(motion["CC_Base_L_Foot"])[:,[2,0,1]]
# l_toe = np.array(motion["CC_Base_L_ToeBase"])[:,[2,0,1]]

# r_thigh = np.array(motion["CC_Base_R_Thigh"])[:,[2,0,1]]
# r_calf = np.array(motion["CC_Base_R_Calf"])[:,[2,0,1]]
# r_foot = np.array(motion["CC_Base_R_Foot"])[:,[2,0,1]]
# r_toe = np.array(motion["CC_Base_R_ToeBase"])[:,[2,0,1]]


skeleton = np.array(
    [
        pelvis,
        l_thigh,
        l_calf,
        l_foot,
        l_toe,
        l_upperarm,
        l_forearm,
        l_hand,
        r_thigh,
        r_calf,
        r_foot,
        r_toe,
        r_upperarm,
        r_forearm,
        r_hand,
    ]
)

# skeleton = skeleton.transpose(1,0,2) / 100
skeleton = skeleton.transpose(1,0,2) / 100
num_frames = skeleton.shape[0]

In [104]:
num_frames

245

In [105]:
skeleton[0,0]

array([ 0.60851575,  0.92007931, -2.44167332])

In [115]:
connection = [[0,1], [1,2], [2,3], [3,4], [0,5], [5,6], [6,7], [0,8], [8,9], [9,10], [10,11], [0,12], [12,13], [13,14]]

data = []

frame_idx = 0
joints_1 = go.Scatter3d(
    x=skeleton[frame_idx, :, 0],  
    y=skeleton[frame_idx, :, 1],  
    z=skeleton[frame_idx, :, 2], 
    mode='markers',
    marker={
        'size': 6,
        'opacity': 0.8,
    }
)

for c in connection:
    line_trace = go.Scatter3d(
        x=skeleton[frame_idx, c, 0],
        y=skeleton[frame_idx, c, 1],
        z=skeleton[frame_idx, c, 2],
        mode='lines',
        line=dict(color='red', width=4)
    )
    data.append(line_trace)

frame_idx = 100
joints_2 = go.Scatter3d(
    x=skeleton[frame_idx, :, 0],  
    y=skeleton[frame_idx, :, 1],  
    z=skeleton[frame_idx, :, 2], 
    mode='markers',
    marker={
        'size': 6,
        'opacity': 0.8,
    }
)

for c in connection:
    line_trace = go.Scatter3d(
        x=skeleton[frame_idx, c, 0],
        y=skeleton[frame_idx, c, 1],
        z=skeleton[frame_idx, c, 2],
        mode='lines',
        line=dict(color='red', width=4)
    )
    data.append(line_trace)

layout = go.Layout(
        margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
        scene = dict(
            # xaxis = dict(title="x", range = [-2,2]),
            # yaxis = dict(title="y", range = [-2,2]),
            # zaxis = dict(title="z", range = [-2,2]),
            aspectratio=dict(x=1, y=1, z=1)
    ))

obstacle = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], color='lightpink', opacity=0.50)


fig = go.Figure(data=data + [joints_1, joints_2, obstacle], layout=layout)
# plotly.offline.iplot(fig)
fig.show()

In [97]:
show_motion(256)

NameError: name 'show_motion' is not defined

In [9]:
robot_urdf = "/home/jianrenw/Research/humanoid_parkour/retarget/h1_description/urdf/h1.urdf"

physicsClient = p.connect(p.DIRECT)  # or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF
robot_start_pos = [0, 0, 0]
robot_start_orientation = p.getQuaternionFromEuler([0, 0, 0])

humanoid = p.loadURDF(robot_urdf, robot_start_pos, robot_start_orientation)

time_step = 1.0 / 240.0
p.setTimeStep(time_step)

num_joints = p.getNumJoints(humanoid)

joint_names = []
joint_lower_limits = []
joint_upper_limits = []

# Iterate through the joints and print their names
for joint_index in range(num_joints):
    joint_info = p.getJointInfo(humanoid, joint_index)
    joint_name = joint_info[1].decode(
        "utf-8"
    )  # Decode the byte string to get the joint name
    joint_lower_limit = joint_info[8]  # Lower joint limit
    joint_upper_limit = joint_info[9]  # Upper joint limit
    joint_names.append(joint_name)
    joint_lower_limits.append(joint_lower_limit)
    joint_upper_limits.append(joint_upper_limit)

joint_lower_limits = np.array(joint_lower_limits)
joint_upper_limits = np.array(joint_upper_limits)

p.disconnect()

b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
imu_linkb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
logo_link

In [10]:
def create_joint_frame_visualization(robot, joint_index, frame_size=0.1):
    joint_position, joint_orientation = p.getLinkState(robot, joint_index)[4:6]


    # Calculate the rotation matrix from Euler angles
    rot_matrix = p.getMatrixFromQuaternion(joint_orientation)
    rot_matrix = np.array(rot_matrix).reshape(3, 3)
    # Extract the rotated axes
    x_axis_end = np.dot(rot_matrix, [frame_size, 0, 0]) + joint_position
    y_axis_end = np.dot(rot_matrix, [0, frame_size, 0]) + joint_position
    z_axis_end = np.dot(rot_matrix, [0, 0, frame_size]) + joint_position


    p.addUserDebugLine(joint_position, x_axis_end, [1, 0, 0], parentObjectUniqueId=-1, parentLinkIndex=-1)

    # Y-axis (green)
    p.addUserDebugLine(joint_position, y_axis_end, [0, 1, 0], parentObjectUniqueId=-1, parentLinkIndex=-1)

    # Z-axis (blue)
    p.addUserDebugLine(joint_position, z_axis_end, [0, 0, 1], parentObjectUniqueId=-1, parentLinkIndex=-1)

# Function to update the base position at each time step
def ik(humanoid):

    joint_poses = []

    init_pose = np.zeros(num_joints)

    for joint_index, angle in enumerate(init_pose):
        p.resetJointState(humanoid, joint_index, angle)
    
    l_shoulder_pos, _ = p.getLinkState(
        humanoid, joint_names.index("left_shoulder_roll_joint")
    )[4:6]
    r_shoulder_pos, _ = p.getLinkState(
        humanoid, joint_names.index("right_shoulder_roll_joint")
    )[4:6]
    l_hip_pos, _ = p.getLinkState(humanoid, joint_names.index("left_hip_pitch_joint"))[4:6]
    r_hip_pos, _ = p.getLinkState(humanoid, joint_names.index("right_hip_pitch_joint"))[4:6]

    for i in range(frame_num):
        
        # IK for l hand
        l_elbow_pos = np.array(l_shoulder_pos) + l_upperarm_dir[i] * upperarm_len
        
        # IK for r hand
        r_elbow_pos = np.array(r_shoulder_pos) + r_upperarm_dir[i] * upperarm_len

        # IK for l foot
        l_ankle_pos = np.array(l_hip_pos) + l_thigh_dir[i] * thigh_len + l_calf_dir[i] * calf_len

        # IK for r foot
        r_ankle_pos = np.array(r_hip_pos) + r_thigh_dir[i] * thigh_len + r_calf_dir[i] * calf_len

        # # Calculate the IK solution
        ik_solution_rh = p.calculateInverseKinematics(
            humanoid,
            joint_names.index("right_elbow_joint"),
            r_elbow_pos,
            targetOrientation = r_elbow_rot[i],
            jointDamping=[0.1] * num_joints,  # Adjust damping as needed
            lowerLimits=joint_lower_limits,
            upperLimits=joint_upper_limits,
            jointRanges=[j_upper - j_lower for j_upper, j_lower in zip(joint_upper_limits, joint_lower_limits)],
            restPoses=[0.0] * num_joints,  # Initial joint angles
        )

        ik_solution_lh = p.calculateInverseKinematics(
            humanoid,
            joint_names.index("left_elbow_joint"),
            l_elbow_pos,
            targetOrientation = l_elbow_rot[i],
            jointDamping=[0.1] * num_joints,  # Adjust damping as needed
            lowerLimits=joint_lower_limits,
            upperLimits=joint_upper_limits,
            jointRanges=[j_upper - j_lower for j_upper, j_lower in zip(joint_upper_limits, joint_lower_limits)],
            restPoses=[0.0] * num_joints,  # Initial joint anglesss
        )

        ik_solution_rf = p.calculateInverseKinematics(
            humanoid,
            joint_names.index("right_ankle_joint"),
            r_ankle_pos,
            targetOrientation = r_ankle_rot[i],
            jointDamping=[0.1] * num_joints,  # Adjust damping as needed
            lowerLimits=joint_lower_limits,
            upperLimits=joint_upper_limits,
            jointRanges=[j_upper - j_lower for j_upper, j_lower in zip(joint_upper_limits, joint_lower_limits)],
            restPoses=[0.0] * num_joints,  # Initial joint angles
        )

        ik_solution_lf = p.calculateInverseKinematics(
            humanoid,
            joint_names.index("left_ankle_joint"),
            l_ankle_pos,
            targetOrientation = l_ankle_rot[i],
            jointDamping=[0.1] * num_joints,  # Adjust damping as needed
            lowerLimits=joint_lower_limits,
            upperLimits=joint_upper_limits,
            jointRanges=[j_upper - j_lower for j_upper, j_lower in zip(joint_upper_limits, joint_lower_limits)],
            restPoses=[0.0] * num_joints,  # Initial joint angles
        )

        # p.resetJointState(humanoid, joint_index, ik_solution)

        ik_solution = np.zeros(num_joints)
        ik_solution[:5] = ik_solution_lf[:5]
        ik_solution[5:10] = ik_solution_rf[5:10]
        ik_solution[11:15] = ik_solution_lh[11:15]
        ik_solution[15:19] = ik_solution_rh[15:19]

        # ik_solution = np.clip(ik_solution, joint_lower_limits, joint_upper_limits)
        # Set the joint angles to achieve the IK solution
        # print(ik_solution)
        for joint_index, angle in enumerate(ik_solution):
            p.resetJointState(humanoid, joint_index, angle)
        
        # create_joint_frame_visualization(humanoid, joint_names.index("left_hand_link"))
        # create_joint_frame_visualization(humanoid, joint_names.index("right_hand_link"))
        # create_joint_frame_visualization(humanoid, joint_names.index("left_ankle"))
        # create_joint_frame_visualization(humanoid, joint_names.index("right_ankle"))

        # Step the simulation
        p.stepSimulation()
        # time.sleep(0.5)
        # print(i)
        joint_poses.append(ik_solution)
    p.disconnect()
    return joint_poses

In [11]:
robot_urdf = "/home/jianrenw/Research/humanoid_parkour/retarget/h1_description/urdf/h1.urdf"

physicsClient = p.connect(p.GUI)  # or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF
robot_start_pos = [0, 0, 0]
robot_start_orientation = p.getQuaternionFromEuler([0, 0, 0])

humanoid = p.loadURDF(robot_urdf, robot_start_pos, robot_start_orientation)

# Start the base position update loop
joint_poses = ik(humanoid)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA Graphics Device/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 520.61.05
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 520.61.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA Graphics Device/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagona

## Save Video

In [12]:
robot_urdf = "/home/jianrenw/Research/humanoid_parkour/retarget/h1_description/urdf/h1.urdf"

physicsClient = p.connect(p.GUI)  # or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath())  # used by loadURDF
robot_start_pos = [0, 0, 0]
robot_start_orientation = p.getQuaternionFromEuler([0, 0, 0])

humanoid = p.loadURDF(robot_urdf, robot_start_pos, robot_start_orientation)

video_filename = "simulation_h1_video.mp4"
frame_width = 1280  # Adjust the frame width as needed
frame_height = 720  # Adjust the frame height as needed
frame_rate = 30  # Adjust the frame rate (frames per second) as needed


time.sleep(5)

p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4, video_filename)

for i in range(frame_num):
    key_t = translation[i] / 100
    key_r = base_rot[i]
    p.resetBasePositionAndOrientation(humanoid, key_t[0].tolist(), key_r.tolist())
    for joint_index, angle in enumerate(joint_poses[i]):
        p.resetJointState(humanoid, joint_index, angle)
    p.stepSimulation()
    time.sleep(5.0 / frame_rate)


p.stopStateLogging(p.STATE_LOGGING_VIDEO_MP4)
p.disconnect()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA Graphics Device/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 520.61.05
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 520.61.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA Graphics Device/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagona

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore destroyed


In [10]:
np.array(joint_poses)

array([[ 0.16987518,  0.01599213, -0.34437726, ...,  0.06131537,
         0.        ,  0.        ],
       [ 0.19750815,  0.02771593, -0.55568496, ..., -0.09586451,
         0.        ,  0.        ],
       [ 0.19737979,  0.02682791, -0.61939508, ..., -0.09432242,
         0.        ,  0.        ],
       ...,
       [ 0.28426226, -0.16334513, -0.49797419, ...,  0.18871116,
         0.        ,  0.        ],
       [ 0.2775931 , -0.17256745, -0.45473287, ...,  0.20171518,
         0.        ,  0.        ],
       [ 0.27887362, -0.17523028, -0.43649568, ...,  0.20163231,
         0.        ,  0.        ]])